# OpenAlex Cited References
### [STI 2023](https://www.sti2023.org/) Conference - Leiden, Netherlands, September 27-29, 2023
#### Eric Schares, Iowa State University; [eschares.github.io](eschares.github.io) 
#### Sandra Mierz; [https://github.com/smierz](https://github.com/smierz) 
---

## Part 2: Graph and explore Citation Data
In this notebook, we will sort through the publication and reference records we collected from OpenAlex in Part 1.

### This notebook expects the following files:
- /files/publications.csv
- /files/pub2ref.csv
- /files/references.csv or .parquet (smaller file size)

In [ ]:
# needed dependencies
import pandas as pd
import plotly.express as px
import pyarrow

---
### 1. Load the publications, references, and connecting data we obtained from Part 1

First we need to load the data. By default, Part 1 saved this in a folder called `files`.

There are two file types:
* `.csv`, or comma separated value. These are human readable and can be opened in programs like Excel, but filesizes can become large
* `.parquet`. These are not human readable or able to be opened in Excel, but the filesizes are smaller and more efficient

We use the `.parquet` file type when saving off the `references` information. For comparison, in our example `references.csv` would be 29MB, while `references.parquet` is less than half the size, at 13MB.

In [ ]:
# Load one year's worth of data
pubs_df = pd.read_csv('../files/ISU_2021_fullyear/publications.csv')
pub2ref_df = pd.read_csv('../files/ISU_2021_fullyear/pub2ref.csv')

#refs_df = pd.read_csv('../files/ISU_2021_fullyear/references.csv')
refs_df = pd.read_parquet('../files/ISU_2021_fullyear/references.parquet')

#### Joining
Join the tables on id fields. This is why we needed to keep unique OpenAlex IDs in Part 1. First drop duplicate pubs and refs...

In [ ]:
### prerequisite for joining: no duplicates in publications or references !

pubs_df.drop_duplicates(inplace=True)
refs_df.drop_duplicates(inplace=True)

Then take the `pub2ref` crosswalk file and add repeated information about the parent pub to the right side

The first two columns are `pub2ref`, rest are from `pubs_df`.

Repeat the parent information over and over so we can keep track of what publication cited what reference

In [ ]:
df = pub2ref_df.join(pubs_df.set_index('publication_id'), on='publication_id')
df.head(3)

A little more pre-processing (rearranging columns, calculating the year delta), and we're ready to investigate!

In [ ]:
#rearrange some things
pub_id_col = df.pop('reference_id') # move reference_id column to end
df['reference_id'] = pub_id_col     # move reference_id column to end

# bring in reference information for each row
df = df.join(refs_df.set_index('reference_id'), on='reference_id')

# Calculate the year delta, or how many years old a reference was when it got cited
df['year_delta'] = df['publication_year'] - df['reference_year']
df.head(3)

In [ ]:
# Save combined table
# Optional, may be a large file, turn on if wanted

#df.to_csv('../files/combined.csv')

### 2. Basic stats about dataframe

Now our data is loaded in. Let's get some basic information:

In [ ]:
print(f"This dataset contains {pubs_df.shape[0]} publications and their {df.shape[0]} cited references.")

In [ ]:
# rows and columns of the combined dataset
df.shape

In [ ]:
# summary statistics by column
df.describe()

In [ ]:
# table with multiindex - connection pub to ref visualized
df_grouped = df.set_index(['publication_id',
                            'publication_doi',
                            'publication_year',
                            'publication_title',
                            'publication_journal',
                            'publication_publisher',
                            'publication_journal_issn',
                            'num_cited_references',
                            'reference_id'])
df_grouped.head(3)

---
### 3. Look at publication ("parent") data

Let's look at the publications we pulled, and sort them by the number of references they contain. You may have to scroll all the way to the right to see the number of references.

In [ ]:
pubs_df = pubs_df.sort_values(by='num_cited_references', ascending=False)
pubs_df = pubs_df.reset_index(drop=True)
pubs_df.head(3)

Wow, one publication has 4,075 cited references!

### Average and median number of references per paper

Let's look more systematically at the number of references per paper

In [ ]:
pubs_df.describe()

We can see that 4,075 number is the max count of references in our dataset. We can also see the 75%, 50%, and 25% quantile counts.

In [ ]:
# Number of references in each unique paper
df['publication_doi'].value_counts()

After 4,075, the next highest number of references is 505.

---
Let's count the papers by number of references:

In [ ]:
pubs_df['num_cited_references'].value_counts().sort_index()

Hmmm, that's interesting...almost 400 papers in this dataset have **no** reported references!

### 4. Look at the 0 reference papers

OpenAlex reports 0 references for some papers, even though manual investigation shows there are references there. 

This is because not all publishers make their reference data openly available. OpenAlex is only able to report on references it knows about. See the **Initiative for Open Citations** [I4OC](https://i4oc.org/) for more information.

In [ ]:
# number of publications with 0 reported references
pubs_df.loc[pubs_df['num_cited_references']==0].shape[0]

In [ ]:
# percentage of publications with 0 reported references
print(f"{(pubs_df.loc[pubs_df['num_cited_references']==0].shape[0] / pubs_df.shape[0])*100:.2f}% of publications in this set have 0 references reported")

---
Time to start making some graphs. Let's add color coding so we can tell what's going on more easily.

In [ ]:
# make all numbers same color except for 0 references
color_dict = {num:'blue' for num in pubs_df['num_cited_references'] if num != 0}
color_dict[0]='lightgray'

Plot a histogram of the number of references a paper has.

The one paper with 4,075 references will skew this plot way to the right, but we can also see the number of 0 reference papers, stacked on the 0 bin and colored light gray.

In [ ]:
fig = px.histogram(pubs_df, x='num_cited_references', nbins=50,
             color='num_cited_references',
             color_discrete_map=color_dict,
             title=f'Histogram of the Number of Cited References in {pubs_df.shape[0]} Publications<br>Num_references=0 shown in light gray'
)
fig.update_layout(showlegend=False)

Now let's plot the same data, but as a cumulative distribution function. Again, the large outlier skews the data to the right.

The good news is, **these plots are all interactive**! You can click and drag a box to zoom in on an area of interest, or pan around by dragging on the axes.

Try zooming in on the range from 0-300 to see what's going on more clearly.

Double click anywhere on the plot to reset the view.

In [ ]:
px.ecdf(pubs_df, x='num_cited_references', ecdfnorm='percent',
       title=f'Cumulative Distribution of the Number of Cited References in {pubs_df.shape[0]} Publications')

#### Remove publications with 0 reported references
Since we have some publications that report no references, let's remove them from our analysis.

We can create another dataframe that matches the criteria of `num_cited_references` not equal to 0.

In [ ]:
pubs_df_no_zeros = pubs_df.loc[pubs_df['num_cited_references']!=0]
pubs_df_no_zeros.head(3)

The statistics of this new dataframe should not show 0 references anywhere. And indeed, we see that the `min` is now 1!

In [ ]:
pubs_df_no_zeros.describe()

Plot the same two graphs again, only this time you should see **no** gray boxes since we removed those 0 reference results.

In [ ]:
px.histogram(pubs_df_no_zeros, x='num_cited_references', nbins=50,
             text_auto=True,
             title=f'Histogram of the Number of Cited References in {pubs_df_no_zeros.shape[0]} Publications<br>Num_references=0 *removed*')

In [ ]:
px.ecdf(pubs_df_no_zeros, x='num_cited_references', ecdfnorm='percent',
       title=f'Cumulative Distribution of the Number of Cited References in {pubs_df_no_zeros.shape[0]} Publications<br>Num_references=0 *removed*',
       width=750)



---
### 5. What journals were cited by these publications?

Let's start looking more deeply at the cited references themselves.

In this dataset, 11,541 unique journals were cited 142,135 times. Why is that number different from the 142,961 total we saw earlier? 826 records have the journal name blank!

In [ ]:
# Number of journals
df['reference_journal'].describe()

How does that break down by journal title?

In [ ]:
journal_counts = df.value_counts(['reference_journal', 'reference_publisher'])
journal_counts = journal_counts.to_frame().reset_index().rename(columns= {0: 'citations', 'reference_journal' : 'title'})
journal_counts.index.name = 'index'

# Top 10 most cited journals
journal_counts.head(10)

Save the top 10 journal counts out to a file

In [ ]:
journal_counts.head(10).to_csv("../files/ISU_2021_fullyear/journal_counts.csv")

Create a bar chart of the top 50 cited journals by number, organized by publisher>

In [ ]:
px.bar(journal_counts[:50], 
       x="title", y="citations", color="reference_publisher",
       height=900,
       color_discrete_sequence=px.colors.qualitative.Set1,
       title=f"Top 50 journals cited in this set, by Title and Publisher")

If you have a specific journal of interest, we can look at *when* the journal was cited? Do our researchers use current material from this journal, or older material?

Let's look at *Physical Review Letters* as an example. We can see from the table above it was the 6th most cited journal, with 1214 references in this set.

In [ ]:
tracked_journal = "Physical Review Letters"

filt = (df['reference_journal']==tracked_journal)
df.loc[filt,'reference_year'].shape

In [ ]:
df_one_journal = df.loc[filt,['reference_journal','reference_year']]
df_one_journal

In [ ]:
fig_one_journal = px.histogram(df_one_journal, x="reference_year", nbins=400, 
             title=f"Histogram of {df_one_journal.shape[0]} Cited References from {tracked_journal}",
                              width=650)

fig_one_journal['layout']['xaxis']['autorange'] = "reversed"

fig_one_journal

Looks like most references were to material from 2021-1996, with few references to pre-1996 content.

We could also look at CDF curves for some top journal titles.

In [ ]:
journal_list = journal_counts['title'].head(8)

journal_list

If a row has `reference_journal` of one of the top 8 titles, keep it an build a new dataframe, `top_journals_by_year`. Then we shorten the journal names since some are very long and mess with the way the legend displays.

You might see a pink warning box when running this cell. That is okay.

In [ ]:
top_journals_by_year = df[df['reference_journal'].isin(journal_list)]

top_journals_by_year['reference_journal_short_name'] = [x[:47] for x in top_journals_by_year['reference_journal']]

fig_top_journals_by_year = px.ecdf(top_journals_by_year, x='reference_year', ecdfnorm='percent',markers=True, lines=False,
        color_discrete_sequence=px.colors.qualitative.Set1,
        color = 'reference_journal_short_name',
        title=f'8 most cited journals, by year published',
        width=950, height=600
)
fig_top_journals_by_year['layout']['xaxis']['autorange'] = "reversed"

fig_top_journals_by_year.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.56
))

fig_top_journals_by_year.show()

We can see three titles (PLOS ONE, Nature Communications, and Scientific Reports) are not referenced prior to ~2006. These are newer journals, and didn't exist during those years!

Physical Review Letteers is in brown, and you can see the large jump at 1996.

Journal of ACS (pink) has the highest tail, meaning more material in that journal was cited from older years. If you hover on 1980, for example, 8.3% of the references to this journal were from 1980 or older.

---
### 6. What publishers were cited?
Now roll up the journal titles into publisher groups. We have 139,817 records with a publisher filled in, and 1,381 of those are unique.

In [ ]:
# Number of publishers
df['reference_publisher'].describe()

Group by publisher name, and organize by number of citations and percentage of this dataset's total.

In [ ]:
publisher_counts = df.value_counts(['reference_publisher'])
publisher_counts = publisher_counts.to_frame().reset_index().rename(columns= {0: 'citations'})
publisher_counts.index.name = 'index'
publisher_counts['percentage'] = (publisher_counts['citations'] / df.shape[0]) * 100

#preview
publisher_counts.head(20)

Plot by counts (a) and percentage (b).

In [ ]:
px.histogram(publisher_counts[:20], 
       x="reference_publisher", y="citations", color="reference_publisher", text_auto=True,
       height=900,
       title=f"Publishers cited in this set")

In [ ]:
px.histogram(publisher_counts[:20], 
       x="reference_publisher", y="percentage", color="reference_publisher",
       height=900,
       title=f"Publishers cited in this set")

---
### 7. What articles were cited?
Let's now turn to look at the individual articles themselves. We have 142,496 records, of whic 123,381 are unique. The most frequent article was cited 59 times in this dataset.

In [ ]:
# Number of articles
df['reference_title'].describe()

Expand beyond the most frequent article, to show the top 10 most cited articles.

In [ ]:
article_counts = df.value_counts(['reference_doi', 'reference_title', 'reference_journal'])
article_counts = article_counts.to_frame().reset_index().rename(columns= {0: 'citations', 'reference_title' : 'reference_title'})
article_counts.index.name = 'index'

#preview
article_counts.head(10)

---
#### What years were those referenced articles published?

The oldest reference in this dataset is from 1690!

In [ ]:
df.loc[df['reference_year']==df['reference_year'].min()]

In [ ]:
df.loc[df['reference_year']==df['reference_year'].min()]['reference_year']

Look at the year the references were published, **by count:**

In [ ]:
fig_years_count = px.histogram(df, x='reference_year', nbins=400, 
             title=f'Histogram of Cited Year<br>{pubs_df.shape[0]} Publications and {df.shape[0]} References')

fig_years_count['layout']['xaxis']['autorange'] = "reversed"

fig_years_count

...by **percentage** (remember, you can click and drag a box to zoom in)

In [ ]:
fig_years_prob = px.histogram(df, x='reference_year', nbins=400, histnorm='probability density',
            title=f'Probability Density of the Cited Year<br>{pubs_df.shape[0]} Publications and {df.shape[0]} References<br>Zoom in')

fig_years_prob['layout']['xaxis']['autorange'] = "reversed"

fig_years_prob.show()

...or by **cumulative distribution.**

In [ ]:
fig_years_cdf = px.ecdf(df, x='reference_year', ecdfnorm='percent',markers=True, lines=False,
        color_discrete_map={'red':'red', 'blue':'blue'},
       title=f'Cumulative Distribution of Year of Citation<br>{df.shape[0]} references'
)
fig_years_cdf['layout']['xaxis']['autorange'] = "reversed"

fig_years_cdf.update_layout(showlegend=False)

---
### 8. Track one publication of interest and its references

Add the `color` column to control the colors. Set it to blue everywhere, then change it to red on records that match the DOI to track.

In this example, I already know of an interesting DOI to track

In [ ]:
# Change DOI in this line
red_doi = 'https://doi.org/10.5252/adansonia2021v43a8'

#https://doi.org/10.5252/adansonia2021v43a8   # publication with oldest average reference at 138 years
#https://doi.org/10.1021/acs.accounts.0c00741 # random example
#https://doi.org/10.1386/ijia_00033_1  - 0 average year, referenced 1 work, which is itself?

df['color'] = 'blue'
red_title = df.loc[df['publication_doi']==red_doi, 'publication_title'].iloc[0]
red_title

In [ ]:
# Change color for that DOI to red
filt = (df['publication_doi'] == red_doi)
df.loc[filt,'color'] = 'red'

Double check the number that you changed to red, should match the number of references in that DOI

In [ ]:
# what does the color column contain?
df['color'].value_counts()

In [ ]:
# how many got changed to red? This will show up in the plot title
red_count = df[df['color']=='red']
red_count.shape[0]

In [ ]:
# how many references did your DOI have? These numbers should match
filt = (df['publication_doi']==red_doi)
df[filt].shape[0]

Plot a histogram, though you probably won't be able to see the red blocks stacked on top of the blue columns very well.

In [ ]:
fig_trackone_count = px.histogram(df, x='reference_year', color='color', nbins=200,
             title=f'Years when Cited References were published<br>Red: "{red_title}"',
             hover_data={'color':False,
                         'reference_title':True},
             color_discrete_map={'red':'red', 'blue':'blue'},
             category_orders={"color":['blue','red']}
)
fig_trackone_count.update_layout(showlegend=False)
fig_trackone_count['layout']['xaxis']['autorange'] = "reversed"

fig_trackone_count.show()

Make a box plot, easier to see the red DOI, but very memory intensive and may slow your machine down. Included here if you want it.

In [ ]:
# fig_trackone_box = px.box(df, x='reference_year', points='all', color='color', notched=True,
#        title=f'Years when Cited References were published<br>Red: "{red_title}"',
#        hover_data={'color':False,
#                     'reference_title':True,
#                    'publication_year':True,      
#                    'publication_title':True},
#        color_discrete_map={'red':'red', 'blue':'blue'},
#        category_orders={"color":['blue','red']}
# )
# fig_trackone_box.update_layout(showlegend=False)
# fig_trackone_box['layout']['xaxis']['autorange'] = "reversed"

# fig_trackone_box.show()

Make a CDF plot (**my personal favorite way to look at this**)

In [ ]:
fig_trackone_ecdf = px.ecdf(df, x='reference_year', color='color', ecdfnorm='percent',markers=True, lines=False,
        color_discrete_map={'red':'red', 'blue':'blue'},
               hover_data={'color':False,
                    'reference_title':True,
                   'reference_year':True},
       title=f'Blue: Cumulative Distribution of Year for {df.shape[0] - red_count.shape[0]} references<br>Red: {red_count.shape[0]} references in "{red_title}"'
)
fig_trackone_ecdf.update_layout(showlegend=False, width=650)

fig_trackone_ecdf['layout']['xaxis']['autorange'] = "reversed"

fig_trackone_ecdf.show()

---
### 9. Year delta

Look at the year delta, or how many years old a reference was when it got cited. This was calculated at the beginning of this notebook after the data load.

This describes each individual journal article referenced in this set.

In [ ]:
df['year_delta'].describe()

Next, group by **publication**, get one number that shows the average age of its references, `avg_year_delta`.

In [ ]:
#df2 = df.groupby('publication_title')['year_delta'].mean().to_frame(name='avg_year_delta')
df2 = df.groupby(['publication_title']).agg(avg_year_delta=('year_delta', 'mean'), num_references=('reference_title', 'size')).reset_index()

df2.sample(5)  # show 5 random results

Sort by the smallest average year delta. Interestingly, some publications have an **average year delta of 0**! This means the paper only cited articles from the same year as itself.

In [ ]:
df2 = df2.sort_values(by='avg_year_delta')
df2

Summary statistics for the `average year delta`. 50% of papers cite material of an average age of 11 years or less.

In [ ]:
df2['avg_year_delta'].describe()

In [ ]:
px.ecdf(df2, x='avg_year_delta',
       title=f'Cumulative Distribution of the Average Age of Reference by Publication<br>{df2.shape[0]} Publications<br>'
)

Save the average year deltas off to a `.csv` file.

In [ ]:
df2.to_csv('../files/avg_year_deltas.csv', index=False)

## That's all for now!
There are almost infinite number of questions you could ask about this data, ways to sort, things to plot, etc. We've only scratched the surface by trying to provide a template for the most common questions.

If you have things you'd like to see, or questions you want answered, let us know.